In [216]:
# from sympy import *
# from sympy.geometry import *
# import sympy as sp
import numpy as np
from scipy.optimize import fsolve,root,brentq,minimize
from TCSPC import *

mono exp phasor = 
$$z(a,n) = \frac{1+i(na)}{1+(na)^2}$$
where $a = \omega \tau$, fundamental angular frequency $\omega = 2\pi T/ N$, $N$ is number of bins , $T$ is period (in ns) $n$ denotes $n^{\text{th}}$ harmonic ($\omega_n = n\omega$).

Under inversion $$T(z) := \left(\frac{1}{z-1}+1\right)^*$$
mono exp phasors become:
$$T(z(a)) = \frac{i}{a}$$

For given bi-exp phasors:
$$z (a_1,a_2) = f \cdot z(a_1) + (1-f)\cdot z(a_2) $$

In [303]:
EGFP = Phasor([0.497,0.503],[2.43,3.07])

In [440]:
p(1)

(0.5679816065500891+0.4920326600625657j)

### Numerical results

In [605]:
N = 19
tau =np.array([2.43,3.07] ) #lifetimes,ns
A = np.array([0.497,0.503]) #amplitudes
a1_true = EGFP.w[1]*tau[0] #mono exp phasor a
a2_true = EGFP.w[1]*tau[1]
f = A*tau/np.sum(A*tau)         #fractional intensity
mono = lambda n,a : (1+n*a*1j)/(1+(n*a)**2)
p_noise = 0.0001*np.random.randn(1, 2).view(np.complex128)[0][0]
p = lambda n : f[0] * mono(n,a1_true) + f[1]*mono(n,a2_true) + p_noise  #add gaussian noise
#p = lambda n : EGFP.phasor[n]

def inv(z):
    '''Perform inversion on a complex number z using circle of inversion at 1 with radius 1'''
    return np.conjugate(1/(z-1)+1)

# def y(x,n):
#     '''Calculates the inverted intercept y of harmonics n for given guess inverted phasor x and bi-exp phasor p
#        Using "angles of the same sgement", angle theta between x,p,1 (centre of inversion circle) and x,y,1 are the same
#        y = tan(theta-pi/2)'''
#     theta = np.angle((x/n*1j-inv(p(n)))/(1-inv(p(n)))) #angle between x,p,1
#     return np.tan(theta-np.pi/2)

def y_intercept(z1,z3,z2= 1):
    '''y-intercept of circle defined by z1,z2,z3,
       z1 lies on y axis (inverted mono exp component)
       z2 = 1
       z3 is the inverted bi exp phasor
       if z1.imag >z3.imag, y-intercept is below z1, hence the np.sign is used
       circle of the form z = (a+bt)/(c+dt) where t is a parameter'''
    a = z2*(z1-z3)
    b = z1*(z2-z3)
    c = z1-z3
    d = z2-z3
    centre = (a*np.conjugate(d)-b*np.conjugate(c))/(c*np.conjugate(d)-d*np.conjugate(c))
    y_sol = z1.imag-2*(z1.imag-centre.imag)
    return y_sol

def y_intercept_slope(z1,z3,alpha):
    '''intercepts of circle with line of slope -cot(alpha), given circle defined by z1,z3,and z2 =1 '''
    #rotate the set up by alpha so the line becomes y intercept
    z2 = np.exp(-alpha*1j)
    z1 *=np.exp(-alpha*1j)
    z3 *=np.exp(-alpha*1j)
    y_sol = y_intercept(z1,z3,z2) #real value only (z4 = y_sol*1j)
    return  np.exp(alpha*1j)*y_sol*1j #rotate back


def y(x,n):
    '''Return y intercept for given x (guess inverted phasor) and harmonic n'''
    z1 = x*1j/n #inverted mono
    z3 = inv(p(n)) #bi-exp
    return y_intercept(z1,z3)

def x_eq(x):
    '''In general, intercepts of a line from guess phasor to given bi-exp phasor of 
       different harmonics and the universal circle may not be related by harmonics. 
       The goal is to find x (inverted mono exp phasor) such that the inverted intercept y are related by harmonics
       inverted phasors are related by nth harmonics: x_n = x/n. The absolute value is taken for minimization purpose
       '''
    return abs(y(x,1)/2-y(x,2))


z3 = inv(p(1))
z1 = 1.7j
z2 = 1


In [556]:
x_eq(1.05)

0.0027081734631786425

In [557]:
print('a1 inverted: ',1/a1_true)
print('a2 inverted: ', 1/a2_true)

a1 inverted:  1.3099172270937887
a2 inverted:  1.0368400201426407


In [579]:
result = minimize(x_eq,x0 = [1/a1_true+1], bounds = [(0,1000)]) #solve the equations with bounds
result

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1.2256862191861728e-12
        x: [ 1.307e+00]
      nit: 6
      jac: [ 5.040e-02]
     nfev: 106
     njev: 53
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>

In [580]:
a1_result = 1/result.x # convert back to real space
a2_result = 1/y(result.x,1) # convert back to real space
tau1_result = a1_result/EGFP.w[1]
tau2_result = a2_result/EGFP.w[1]
print(tau1_result)
print(tau2_result)

[2.43516122]
[3.08673992]


In [760]:
EGFP.n_photon = int(1e9)
EGFP.repeat_sim(100)

In [762]:
N = 380 #N histogram bins
alpha = 2*np.pi/N
mono_d = lambda x,n :(1-x)/(1-x*np.exp(1j*n*alpha)) #d for discrete

z1 = lambda x, n : inv(mono_d(x,n)) #0.5 * (1-1/x)*(1-1j/np.tan(alpha*n/2))
z3 = lambda n :inv(EGFP.phasor_data[16][n])
x = lambda tau: np.exp(-1/19/tau)
p_d = lambda n : np.sum(A*np.array([mono_d(x(tau[0]),n),mono_d(x(tau[1]),n)])) #pure bi-exp discrete phasor
#z3 = lambda n : inv(p_d(n))
print(y_intercept_slope(z1(x(tau[0]),1),z3(1),alpha/2))
print(y_intercept_slope(z1(x(tau[0]),2),z3(2),alpha))
def discrete_eq(tau):
    return abs(y_intercept_slope(z1(x(tau),1),z3(1),alpha/2).real-y_intercept_slope(z1(x(tau),2),z3(2),alpha).real)
discrete_eq(tau[0])

(-0.008671265396496949+1.048832914307877j)
(-0.00870997617732805+0.5267215833930098j)


3.871078083110026e-05

In [763]:
result_discrete = minimize(discrete_eq,x0 = [3.07+0.001], bounds = [(0,10)])
result_discrete

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 1.7571777366498509e-12
        x: [ 3.160e+00]
      nit: 5
      jac: [ 3.584e-04]
     nfev: 58
     njev: 29
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>

In [764]:
-1/np.log(1/(1-y_intercept_slope(z1(x(result_discrete.x),1),z3(1),alpha/2).real*2))/19

array([2.50755364])

In [765]:
np.exp(-1/19/tau[0])

0.9785737877089958

In [766]:
x_eq(0.5)

0.005062288575523932

In [130]:
result

1.9999999999999933

In [136]:
alpha, u,v,w = 
def harmonic_circ(n):
    '''Returns circle of nth harmonics where different lifetimes lie on
    Input: n -harmonic'''
    return Circle((0.5,-0.5*tan(a)),0.5/cos(a))

def phasor_ray(z,p):
    '''Ray drawn from guess phasor z to given bi-exp phasor p'''
    return Ray((z.real,z.imag),(p.real,p.imag))

In [137]:
a = np.exp(-dt/tau)
def z(n):
    return (1+a)/(1+a*np.exp(1j*n*np.pi/N))
p1 = 0.35+0.16j
p2 = 0.25+0.008j
p = [p1,p2]
def z2(n):
    '''Return z2 for given harmonic'''
    z2 = intersection(harmonic_circ(n),phasor_ray(z(n),p[n-1]))[0].coordinates
    return z2[0]+1j*z2[1]
    

In [138]:
intersection(harmonic_circ(2),phasor_ray(z(2),p[2-1]))[0].coordinates

(12668928146697458581149947214673993512056737/10990916146227808512338131395890000000000000 - 226649235638391*sqrt(23272858502612877813499040969360811714832961960459414278353763641989317233791489)/1152948455874774170327956596802799716575470000000000000,
 -94571119618349517916946188231557916551821707/109909161462278085123381313958900000000000000 + 20787*sqrt(23272858502612877813499040969360811714832961960459414278353763641989317233791489)/109909161462278085123381313958900000000000000)

In [139]:
z2_inv = lambda n : conjugate(1/(z2(n)-1)+1)

In [140]:
def z2_diff(tau):
    '''Return z2_inv(1).real-z2_inv(2).real -> check z2_inv(1) = z2_inv(2)'''
    global a
    a = np.exp(-dt/tau)
    return float(re(z2_inv(1)-z2_inv(2)))

In [148]:
z2_diff(1)

-0.022806951667178155

In [133]:
-0.021410393939246854

-0.0018342873698134643

In [97]:
x = symbols('x')
z_2 = (1+x)/(1+x*exp(1j*2*pi*dt))

In [98]:
from scipy.optimize import root

In [101]:
nsolve(z2_diff,1)

AttributeError: 'function' object has no attribute 'free_symbols'

In [100]:
z(2)

array([1.00000175-0.00805002j])